# Problem set 2: Finding the Walras equilibrium in a multi-agent economy

In [1]:
%load_ext autoreload
%autoreload 2

# Tasks

## Drawing random numbers

Replace the missing lines in the code below to get the same output as in the answer.

In [2]:
import numpy as np
np.random.seed(1986)
# missing line
state = np.random.get_state()
for i in range(3):
    # missing line
    #in each new generating process we go back to the original state
    np.random.set_state(state)
    for j in range(2):
        x = np.random.uniform()
        print(f'({i},{j}): x = {x:.3f}')

(0,0): x = 0.569
(0,1): x = 0.077
(1,0): x = 0.569
(1,1): x = 0.077
(2,0): x = 0.569
(2,1): x = 0.077


A seed sets the point of generating numbers. This means we will get the same numbers each time, as long as we choose the same seed. 

**Answer:**

See A1.py

## Find the expectated value

Find the expected value and the expected variance

$$ 
\mathbb{E}[g(x)] \approx \frac{1}{N}\sum_{i=1}^{N} g(x_i)
$$
$$ 
\mathbb{VAR}[g(x)] \approx \frac{1}{N}\sum_{i=1}^{N} \left( g(x_i) - \frac{1}{N}\sum_{i=1}^{N} g(x_i) \right)^2
$$

where $ x_i \sim \mathcal{N}(0,\sigma) $ and

$$ 
g(x,\omega)=\begin{cases}
x & \text{if }x\in[-\omega,\omega]\\
-\omega & \text{if }x<-\omega\\
\omega & \text{if }x>\omega
\end{cases} 
$$

In [33]:
sigma = 3.14 #standard deviation
omega = 2
N = 10000
np.random.seed(1986)
# write your code here

#defining the g-function: 
def g_func(x, omega):
    if x > omega: 
        return omega
    elif x < -omega: 
        return -omega
    else:
        return x

#another way of doing the g function:    
def g(x, omega):
    y = x.copy()
    y[x < -omega] = -omega
    y[x >  omega] = omega
    return y


#testing the function: 
print(g_func(2, omega))

#finding the expected value
#need to use all the random draws of 10000
#drawing 10000 iterations of a random variable x:
x = np.random.normal(0, sigma, N)
#sum
mysum = 0
for i in range(N):
    mysum += g_func(x[i], omega)
#expectation
expected = mysum/N
#variance
var_sum = 0 
for i in range(N):
    var_sum += (g_func(x[i], omega) - expected)**2
variance = 1/N * var_sum 

print(mysum, expected, variance)



2
-26.41941946664655 -0.0026419419466646547 2.6980407418769548


In [25]:
#more compact solution 
mysum = sum([g_func(x[i], omega) for i in range(N)])
print(mysum)

-26.41941946664655


**Answer:**

See A2.py

## Interactive histogram

**First task:** Consider the code below. Fill in the missing lines so the figure is plotted.

In [26]:
# a. import
%matplotlib inline
import matplotlib.pyplot as plt
# missing line

# b. plotting figure
def fitting_normal(X,mu_guess,sigma_guess):
    
    # i. normal distribution from guess
    F = norm(loc=mu_guess,scale=sigma_guess)
    
    # ii. x-values
    # missing line, x_low =
    # missing line, x_high =
    x = np.linspace(x_low,x_high,100)

    # iii. figure
    fig = plt.figure(dpi=100)
    ax = fig.add_subplot(1,1,1)
    ax.plot(x,F.pdf(x),lw=2)
    ax.hist(X,bins=100,density=True,histtype='stepfilled');
    ax.set_ylim([0,0.5])
    ax.set_xlim([-6,6])

# c. parameters
mu_true = 2
sigma_true = 1
mu_guess = 1
sigma_guess = 2

# d. random draws
X = np.random.normal(loc=mu_true,scale=sigma_true,size=10**6)

# e. figure
try:
    fitting_normal(X,mu_guess,sigma_guess)
except:
    print('failed')

failed


**Second task:** Create an interactive version of the figure with sliders for $\mu$ and $\sigma$.

In [27]:
# write your code here

**Answer:**

See A3.py

## Modules

1. Call the function `myfun` from the module `mymodule` present in this folder.
2. Open VSCode and open the `mymodule.py`, add a new function and call it from this notebook.

In [38]:
# write your code here
import mymodule
mymodule.myfun(1)

hello world!


**Answer:**

See A4.py

## Git

1. Try to go to your own personal GitHub main page and create a new repository. Then put your solution to this problem set in it.
2. Pair up with a fellow student. Clone each others repositories and run the code in them.

**IMPORTANT:** You will need **git** for the data project in a few needs. Better learn it know. Remember, that the teaching assistants are there to help you.

# Problem

Consider an **exchange economy** with

1. 2 goods, $(x_1,x_2)$
2. $N$ consumers indexed by $j \in \{1,2,\dots,N\}$
3. Preferences are Cobb-Douglas with truncated normally *heterogenous* coefficients

    $$
    \begin{aligned}
    u^{j}(x_{1},x_{2}) & = x_{1}^{\alpha_{j}}x_{2}^{1-\alpha_{j}}\\
     & \tilde{\alpha}_{j}\sim\mathcal{N}(\mu,\sigma)\\
     & \alpha_j = \max(\underline{\mu},\min(\overline{\mu},\tilde{\alpha}_{j}))
    \end{aligned}
    $$

4. Endowments are *heterogenous* and given by

    $$
    \begin{aligned}
    \boldsymbol{e}^{j}&=(e_{1}^{j},e_{2}^{j}) \\
     &  & e_i^j \sim f, f(x,\beta_i) =  1/\beta_i \exp(-x/\beta)
    \end{aligned}
    $$

**Problem:** Write a function to solve for the equilibrium.

You can use the following parameters:

Breaking down a problem with smaller functions and optimizations

In [6]:
# a. parameters
import numpy as np

N = 10000
mu = 0.5
sigma = 0.2
mu_low = 0.1
mu_high = 0.9
beta1 = 1.3
beta2 = 2.1
seed = 1986

# b. draws of random numbers
np.random.seed(seed)
alphas = np.random.normal(mu, sigma, size=N)
#output is a vector of size N
#np can be used as operators on arrays -> avoid using a loop
#transforming the alphas
alphas = np.maximum(mu_low, np.minimum(mu_high, alphas))

e1 = np.random.exponential(beta1, N)
e2 = np.random.exponential(beta2, N)

# c. demand function
def demand_good_1_func(alphas, p1, p2, e1, e2):
    I = e1*p1+e2*p2
    return alphas*I/p1

# # d. excess demand function
def excess_demand_good_1_func(alphas, p1, p2, e1, e2):
    
    # a. demand
    demand = np.sum(demand_good_1_func(alphas, p1, p2, e1, e2))
    
    # b. supply
    supply = np.sum(e1)
    
    # c. excess demand
    excess_demand = demand-supply
    
    return excess_demand

# e. find equilibrium function
#maxiter tells the loop to run for a max of 500 times
def find_equilibrium(alphas, p1, p2, e1, e2, kappa=0.5, eps=1e-8, maxiter=500):
    
    t = 0
    #the following loop runs inf
    while True:

        # a. step 1: excess demand
        Z1 = excess_demand_good_1_func(alphas,p1,p2,e1, e2)
        
        # b: step 2: if we reach the limit for our loop just print whatever excess demand and p1 is at this point
        if  np.abs(Z1) < eps or t >= maxiter:
            print(f'{t:3d}: p1 = {p1:12.8f} -> excess demand -> {Z1:14.8f}')
            break    
    
        # c. step 3: update p1
        p1 = p1 + kappa*Z1/alphas.size
            
        # d. step 4: return 
        if t < 5 or t%25 == 0:
            print(f'{t:3d}: p1 = {p1:12.8f} -> excess demand -> {Z1:14.8f}')
        elif t == 5:
            print('   ...')
            
        t += 1    

    return p1

# f. call find equilibrium function
p1 = 1.4
p2 = 1
kappa = 0.1
eps = 1e-8
p1 = find_equilibrium(alphas, p1, p2, e1, e2, kappa=kappa, eps=eps)

  0: p1 =   1.41028127 -> excess demand ->  1028.12662815
  1: p1 =   1.42001183 -> excess demand ->   973.05595602
  2: p1 =   1.42922852 -> excess demand ->   921.66961919
  3: p1 =   1.43796494 -> excess demand ->   873.64220789
  4: p1 =   1.44625180 -> excess demand ->   828.68584410
   ...
 25: p1 =   1.55186938 -> excess demand ->   302.16618183
 50: p1 =   1.59667790 -> excess demand ->   101.85149536
 75: p1 =   1.61209958 -> excess demand ->    35.71520188
100: p1 =   1.61754486 -> excess demand ->    12.69038081
125: p1 =   1.61948436 -> excess demand ->     4.53002946
150: p1 =   1.62017729 -> excess demand ->     1.61971617
175: p1 =   1.62042512 -> excess demand ->     0.57946965
200: p1 =   1.62051379 -> excess demand ->     0.20735439
225: p1 =   1.62054553 -> excess demand ->     0.07420416
250: p1 =   1.62055688 -> excess demand ->     0.02655552
275: p1 =   1.62056095 -> excess demand ->     0.00950355
300: p1 =   1.62056240 -> excess demand ->     0.00340109
325: p1

**Hint:** The code structure is exactly the same as for the exchange economy considered in the lecture. The code for solving that exchange economy is reproduced in condensed form below.

In [30]:
# a. parameters
N = 1000
k = 2
mu_low = 0.1
mu_high = 0.9
seed = 1986

# b. draws of random numbers
np.random.seed(seed)
alphas = np.random.uniform(low=mu_low,high=mu_high,size=N)

# c. demand function
def demand_good_1_func(alpha,p1,p2,k):
    I = k*p1+p2
    return alpha*I/p1

# d. excess demand function
def excess_demand_good_1_func(alphas,p1,p2,k):
    
    # a. demand
    demand = np.sum(demand_good_1_func(alphas,p1,p2,k))
    
    # b. supply
    supply = k*alphas.size
    
    # c. excess demand
    excess_demand = demand-supply
    
    return excess_demand

# e. find equilibrium function
def find_equilibrium(alphas,p1,p2,k,kappa=0.5,eps=1e-8,maxiter=500):
    
    t = 0
    while True:

        # a. step 1: excess demand
        Z1 = excess_demand_good_1_func(alphas,p1,p2,k)
        
        # b: step 2: stop?
        if  np.abs(Z1) < eps or t >= maxiter:
            print(f'{t:3d}: p1 = {p1:12.8f} -> excess demand -> {Z1:14.8f}')
            break    
    
        # c. step 3: update p1
        p1 = p1 + kappa*Z1/alphas.size
            
        # d. step 4: return 
        if t < 5 or t%25 == 0:
            print(f'{t:3d}: p1 = {p1:12.8f} -> excess demand -> {Z1:14.8f}')
        elif t == 5:
            print('   ...')
            
        t += 1    

    return p1

# e. call find equilibrium function
p1 = 1.4
p2 = 1
kappa = 0.1
eps = 1e-8
p1 = find_equilibrium(alphas,p1,p2,k,kappa=kappa,eps=eps)

  0: p1 =   1.33690689 -> excess demand ->  -630.93108302
  1: p1 =   1.27551407 -> excess demand ->  -613.92820358
  2: p1 =   1.21593719 -> excess demand ->  -595.76882769
  3: p1 =   1.15829785 -> excess demand ->  -576.39340748
  4: p1 =   1.10272273 -> excess demand ->  -555.75114178
   ...
 25: p1 =   0.53269252 -> excess demand ->   -53.80455643
 50: p1 =   0.50897770 -> excess demand ->    -0.27125769
 75: p1 =   0.50886603 -> excess demand ->    -0.00120613
100: p1 =   0.50886553 -> excess demand ->    -0.00000536
125: p1 =   0.50886553 -> excess demand ->    -0.00000002
130: p1 =   0.50886553 -> excess demand ->    -0.00000001


**Answers:**

See A5.py

## Save and load

Consider the code below and fill in the missing lines so the code can run without any errors.

In [31]:
import pickle

# a. create some data
my_data = {}
my_data['A'] = {'a':1,'b':2}
my_data['B'] = np.array([1,2,3])
# missing line

my_np_data = {}
my_np_data['D'] = np.array([1,2,3])
my_np_data['E'] = np.zeros((5,8))
# missing line

# c. save with pickle
with open(f'data.p', 'wb') as f:
    # missing line
    pass
    
# d. save with numpy
# missing line, np.savez(?)
    
# a. try
def load_all():
    with open(f'data.p', 'rb') as f:
        data = pickle.load(f)
        A = data['A']
        B = data['B']
        C = data['C']

    with np.load(f'data.npz') as data:
        D = data['D']
        E = data['E']
        F = data['F']        
    
    print('variables loaded without error')
    
try:
    load_all()
except:
    print('failed')

failed


**Answer:**

See A6.py

# Extra Problems

## Multiple goods

Solve the main problem extended with multiple goods:

$$
\begin{aligned}
u^{j}(x_{1},x_{2}) & = x_{1}^{\alpha^1_{j}} \cdot x_{2}^{\alpha^2_{j}} \cdots x_{M}^{\alpha^M_{j}}\\
 &  \alpha_j = [\alpha^1_{j},\alpha^2_{j},\dots,\alpha^M_{j}] \\
 &  \log(\alpha_j) \sim \mathcal{N}(0,\Sigma) \\
\end{aligned}
$$

where $\Sigma$ is a valid covariance matrix.

In [32]:
# a. choose parameters
N = 10000
J = 3

# b. choose Sigma
Sigma_lower = np.array([[1, 0, 0], [0.5, 1, 0], [0.25, -0.5, 1]])
Sigma_upper = Sigma_lower.T
Sigma = Sigma_upper@Sigma_lower
print(Sigma)

# c. draw random numbers
alphas = np.exp(np.random.multivariate_normal(np.zeros(J), Sigma, 10000))
print(np.mean(alphas,axis=0))
print(np.corrcoef(alphas.T))

# write your code here

[[ 1.3125  0.375   0.25  ]
 [ 0.375   1.25   -0.5   ]
 [ 0.25   -0.5     1.    ]]
[1.91709082 1.91100849 1.63670693]
[[ 1.          0.19955924  0.15149459]
 [ 0.19955924  1.         -0.16150109]
 [ 0.15149459 -0.16150109  1.        ]]
